### Sistema de Recomendación de Noticias sobre Clientes con Potencial Comercial

In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

clientes = pd.read_csv('./clientes.csv')
noticias =  pd.read_excel('./noticias.xlsx')
clientes_noticias = pd.read_csv('./clientes_noticias.csv')

In [8]:
cli = clientes_noticias.merge(clientes, how='left', on='nit')
noticias_subsec = cli[['news_id', 'subsec', 'nit']].groupby('news_id').apply(pd.DataFrame.mode).reset_index(drop=True)

In [11]:
df = noticias.merge(noticias_subsec, how='inner', on='news_id')

### Se define en número el número de sectores:

In [12]:
topic_num = 4
init = 10
top_news = list(df.subsec.value_counts()[init:init+topic_num].index)
df_news = df[df.subsec.isin(top_news)].drop_duplicates(subset=['news_title']).reset_index(drop=True).copy()

## Homogenización de texto

In [16]:
!python -m spacy download es_core_news_sm
#!pip install Unidecode
import unidecode
import re
import spacy
nlp = spacy.load("es_core_news_sm")

extra_stopwords = pd.read_csv('extra_stopwords.csv', sep=',',header=None)
extra_stopwords.columns = ['stopwords']
extra_stopwords=set(extra_stopwords['stopwords'].to_list())

nlp.Defaults.stop_words |= extra_stopwords


def text_cleaning(txt):
    out = unidecode.unidecode(txt)
    out = re.sub("[^\\w\\s]|\n", ' ', out)
    out = re.sub("\d+", "", out)
    out = re.sub('\s+', ' ', out)

    out = out.lower()
    #NLP object
    out = nlp(out)
    # Eliminar Stopwords
    out = [token.text for token in out if not token.is_stop]
    out = " ".join(out)
    # Obtener los lemas de cada palabra
    lemmas =[token.lemma_ for token in nlp(out)]
    # Convertir la lista de lemmas nuevamente a texto
    out = " ".join(lemmas)
    # Remover palabras muy cortas
    out = [token.text for token in nlp(out) if len(token) > 2]

    return out

2023-09-27 20:19:11.852398: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 52.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [17]:
clean_txt = list(map(text_cleaning, df_news['news_text_content']))
clean = [ ' '.join(i) for i in clean_txt]

df_news['news_text_content_clean'] = clean

### CountVectorizer

In [18]:
# Import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

#Definimos un objeto CountVectorizer y creamos los vectores
count = CountVectorizer(stop_words=list(nlp.Defaults.stop_words))
count_matrix = count.fit_transform(df_news['news_text_content_clean'])

#Importamos la función
from sklearn.metrics.pairwise import cosine_similarity

#Calculamos la matriz de similitud de coseno
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [ ]:
sim_scores = list(enumerate(cosine_sim2[0]))
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
sim_scores

In [50]:
sim_scores

[(292, 0.28953978319880663), (131, 0.2868970612914725)]

In [51]:
def recomendador(news_title, cosine_sim, df, num_reco):
    # Paso 2
    df = df.reset_index()
    indices = pd.Series(df.index, index=df['news_title']).drop_duplicates()

    # Paso 3
    idx = indices[news_title]

    # Paso 4
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Paso 5
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    #num_reco = 5
    # Paso 6
    sim_scores = sim_scores[1:num_reco+1]

    news_indices = [i[0] for i in sim_scores]
    sim_values = [i[1] for i in sim_scores]

    # Paso 7
    reco_titles = df['news_title'].iloc[news_indices].reset_index(drop=True)
    reco_subsec = df['subsec'].iloc[news_indices].reset_index(drop=True)

    # Crear un dataframe con los títulos y subsec de las noticias recomendadas
    reco_df = pd.DataFrame({'news_title': reco_titles, 'subsec': reco_subsec})

    # Agregar la noticia original y su subsec como nuevas columnas en el dataframe
    reco_df['original_news_title'] = df['news_title'][idx]
    reco_df['original_news_subsec'] = df['subsec'][idx]

    # Agregar los valores de sim_scores como una nueva columna en el dataframe
    reco_df['sim_scores'] = sim_values

    # Agregar una nueva columna que sea 1 si subsec es igual a original_news_subsec, y 0 en caso contrario
    reco_df['match'] = np.where(reco_df['subsec'] == reco_df['original_news_subsec'], 1, 0)

    return reco_df

### Ejemplo de de recomendación para el primer cliente.

Titulo de la noticia:

In [68]:
df_news['news_title'][10]

'Por no contar la verdad, Hugo Aguilar quedo sin cupo en la JEP'

Sector del cliente:

In [53]:
df_news['subsec'][10]

'ELECTRICIDAD'

### Números de recomendaciones:

In [70]:
num_reco = 5

### Recomendaciones:

In [79]:
recomendador(df_news['news_title'][0], cosine_sim2, df_news, num_reco)[['news_title', 'subsec', 'sim_scores']]

,news_title,subsec,sim_scores
0,9 de cada 10 adultos en Colombia contaba con a...,SEGUROS,0.289540
1,Sube de 43% a 57% porcentaje de peruanos con c...,SEGUROS,0.286897
2,Las ventajas de emplear un software de contabi...,TRANSPORTE TERRESTRE,0.256613
3,El 49% de peruanos ya recibe o realiza pagos d...,SEGUROS,0.247232
4,La penetracion de los seguros se mantuvo en 3%...,SEGUROS,0.243572


# Evaluación sobre todas las noticias

In [80]:
scores = []
for i in range(len(df_news)):
    reco = recomendador(df_news['news_title'][i], cosine_sim2, df_news, num_reco)
    scores.append(len(reco[reco['match'] == 1])/num_reco)

In [81]:
df_news['score_CountVectorizer'] = scores

In [82]:
df_news[['subsec', 'score_CountVectorizer']].groupby(by=["subsec"]).mean()

,score_CountVectorizer
subsec,
ELECTRICIDAD,0.525980
MEDIOS,0.487805
SEGUROS,0.541544
TRANSPORTE TERRESTRE,0.477083


**texto en negrita**## TF-IDFVectorizer

In [86]:
from sklearn.feature_extraction.text import TfidfVectorizer

#Definimos el objeto TF-IDF Vectorizer Object
tfidf = TfidfVectorizer()

#Construimos la matriz TF-IDF
tfidf_matrix = tfidf.fit_transform(df_news['news_text_content_clean'].values)

# Convertir la matriz dispersa a un DataFramem con el el vocabulario (palabras) que el TfidfVectorizer está utilizando
df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf.get_feature_names_out())

# Cargamos las funciones a utilizar
from sklearn.metrics.pairwise import linear_kernel

# Calculamos el producto punto
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)



### Ejemplo

In [88]:
recomendador(df_news['news_title'][0], cosine_sim, df_news, num_reco)[['news_title', 'subsec', 'sim_scores']]

,news_title,subsec,sim_scores
0,El 60% de los valencianos consideran que exist...,SEGUROS,0.206347
1,9 de cada 10 adultos en Colombia contaba con a...,SEGUROS,0.196525
2,Por que las empresas y la academia cooperan pa...,SEGUROS,0.179647
3,Sube de 43% a 57% porcentaje de peruanos con c...,SEGUROS,0.174100
4,La educacion gratuita | Columna de Jesus Ferro...,TRANSPORTE TERRESTRE,0.162873


# Evaluación sobre todas las noticias

In [89]:
scores = []
for i in range(len(df_news)):
    reco = recomendador(df_news['news_title'][i], cosine_sim, df_news, num_reco)
    scores.append(len(reco[reco['match'] == 1])/num_reco)

df_news['score_TF_IDFVectorizer'] = scores

In [90]:
df_news[['subsec', 'score_TF_IDFVectorizer']].groupby(by=["subsec"]).mean()

,score_TF_IDFVectorizer
subsec,
ELECTRICIDAD,0.529902
MEDIOS,0.513082
SEGUROS,0.519485
TRANSPORTE TERRESTRE,0.495833


## Recomendaciones basadas en temas


In [91]:
# Utilice este espacio para escribir el código.

# Cargamos la función
from gensim.corpora import Dictionary

# Creamos la representación de diccionario del documento
dictionary = Dictionary(clean_txt)

corpus = [dictionary.doc2bow(doc) for doc in clean_txt]

#print('Numero de palabras únicas: %d' % len(dictionary))

from gensim.models import LdaModel

from gensim.models.ldamulticore import LdaMulticore

from gensim.models import CoherenceModel

In [92]:
model = LdaMulticore(corpus=corpus,
                        id2word=dictionary,
                        num_topics=topic_num,
                        random_state=123,
                        passes=20)

In [93]:
topic = [model.get_document_topics(item)[0][0] for item in corpus]
topic_proba = [model.get_document_topics(item)[0][1] for item in corpus]

In [94]:
df_news['topic'] = topic
df_news['topic_proba'] = topic_proba

In [95]:
df_news[['topic', 'subsec']].groupby(by=['topic', 'subsec' ]).size()

topic  subsec              
0      ELECTRICIDAD            259
       MEDIOS                  207
       SEGUROS                 425
       TRANSPORTE TERRESTRE    207
1      ELECTRICIDAD             60
       MEDIOS                  129
       SEGUROS                  55
       TRANSPORTE TERRESTRE     29
2      ELECTRICIDAD             47
       MEDIOS                   67
       SEGUROS                  43
       TRANSPORTE TERRESTRE     82
3      ELECTRICIDAD             42
       MEDIOS                   48
       SEGUROS                  21
       TRANSPORTE TERRESTRE     66
dtype: int64

In [126]:
#!pip install pyLDAvis
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
!pip install pandas==1.5.3


pyLDAvis.enable_notebook()

pyLDAvis.enable_notebook()
LDA_visualization = gensimvis.prepare(model, corpus, dictionary)
LDA_visualization

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 76.1 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.1.1
    Uninstalling pandas-2.1.1:
      Successfully uninstalled pandas-2.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyldavis 3.4.1 requires pandas>=2.0.0, but you have pandas 1.5.3 which is incompatible.


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.062125  0.096640       1        1  38.442927
3     -0.009641 -0.093137       2        1  26.094003
2     -0.073481 -0.025669       3        1  20.878740
1      0.145247  0.022166       4        1  14.584330, topic_info=          Term         Freq        Total Category  logprob  loglift
300   producto   918.000000   918.000000  Default  30.0000  30.0000
578    mercado  1055.000000  1055.000000  Default  29.0000  29.0000
2238     calle   369.000000   369.000000  Default  28.0000  28.0000
3683     metro   408.000000   408.000000  Default  27.0000  27.0000
217   femenino   199.000000   199.000000  Default  26.0000  26.0000
...        ...          ...          ...      ...      ...      ...
1785  atencion   131.601115   443.802376   Topic4  -6.3148   0.7096
269    mundial   129.408700   442.779962   Topic4  -6.3316   0.6951
621     tiempo   138.904674   798.672781   Topic4  -6.2608   0.1761
79     persona   143.882404  1204.139541   Topic4  -6.2256  -0.1993
581   nacional   135.163527  1115.221359   Topic4  -6.2881  -0.1851

[376 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
25465      3  0.936879      aborto
1741       3  0.962711   acueducto
9858       3  0.947463    acusador
6606       1  0.081819  aeropuerto
6606       2  0.818191  aeropuerto
...      ...       ...         ...
1084       1  0.078962         win
1084       2  0.026321         win
1084       4  0.894900         win
39713      4  0.975653       wraps
29278      3  0.957377        wust

[827 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 3, 2])

In [122]:
def recomendador_lda(news_title, df, num_reco):
  topic_new = df[df.news_title == news_title].topic[0]
  subsec_new = df[df.news_title == news_title].subsec[0]
  df = df[df.subsec == subsec_new]
  df = df[df.news_title != news_title].copy()

  return df[df.topic == topic_new].sort_values(by='topic_proba',
      ascending=False)[['news_title',	'subsec', 'topic_proba']].reset_index(drop=True)[1:num_reco+1]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [124]:
recomendador_lda(df_news.news_title[0], df_news, num_reco)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,news_title,subsec,topic_proba
1,"Ana Paula Marques (EDP): ""Si tenemos una carga...",SEGUROS,0.999086
2,"Economia peruana crecio 2.28% en mayo, se desa...",SEGUROS,0.998983
3,Mafia de combustibles y alianza Marti-Total,SEGUROS,0.998971
4,Volaris reactiva ruta Guanajuato-Merida; Zoho ...,SEGUROS,0.998963
5,MAPFRE gana 338 millones de euros en los seis ...,SEGUROS,0.998932


In [125]:
df_news.news_title[0]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'Javier Rodriguez Soler, nuevo presidente del Consejo Asesor del Centro de Educacion y Capacidades Financieras de BBVA'

In [ ]:
df_news[df_news['news_title'] == 'Por no contar la verdad, Hugo Aguilar quedo sin cupo en la JEP']
recomendador_lda(df_news[df_news['news_title'] == 'Por no contar la verdad, Hugo Aguilar quedo sin cupo en la JEP'], df_news, num_reco)